## NOTEBOOK DESCRIPTION:

This notebook checks the behaviour of the embedding space: 
- It first checks manually the nearest neighbors of some selected channels
- It also compute the user jumper score which is a first barrier on the evaluation of the embedding

In [1]:
import pickle
import os
import sys
import random

import numpy as np
import pandas as pd


scriptpath = "/home/jouven/youtube_projects/"
sys.path.append(os.path.abspath(scriptpath))

from helpers.helpers_channels_more_10k import *
from helpers.helpers_channel_embedding import *

scriptpath = "/home/jouven/youtube_projects"
sys.path.append(os.path.abspath(scriptpath))
from helpers.config_threshold_value import *

### Useful functions

In [2]:
# Selected channels and id-index mapping
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

In [3]:
PATH = '/dlabdata1/youtube_large/jouven/word2vecf_preprocessing/channels_more_10k/channel_embedding_30M_dim_400.npy'

In [4]:
channelcrawler = pd.read_csv("/dlabdata1/youtube_large/df_channels_en.tsv.gz", sep='\t')
# Select the rows being in the select channels
channelcrawler = channelcrawler[channelcrawler['channel'].apply(lambda row: row in channels_id)]

In [5]:
channelcrawler.head()

category_cc   join_date                   channel  \
0         Gaming  2010-04-29  UC-lHJZR3Gqxm24_Vd_AJ5Yw   
1      Education  2006-09-01  UCbCmjCuTUZos6Inko4u57UQ   
2  Entertainment  2006-09-20  UCpEhnqL0y41EpW2TvWAHD7Q   
3  Howto & Style  2016-11-15  UC295-Dw_tDNtZXFeAPAW6Aw   
4         Sports  2007-05-11  UCJ5v_MCY6GNUBTO8-D3XoAg   

                   name_cc  subscribers_cc  videos_cc  subscriber_rank_sb  \
0                PewDiePie       101000000       3956                 3.0   
1  Cocomelon - Nursery ...        60100000        458                 7.0   
2                SET India        56018869      32661                 8.0   
3          5-Minute Crafts        60600000       3591                 9.0   
4                      WWE        48400000      43421                11.0   

   weights  
0    2.087  
1    2.087  
2    2.087  
3    2.087  
4    2.087

In [6]:
channelcrawler['category_cc'].unique()

array(['Gaming', 'Education', 'Entertainment', 'Howto & Style', 'Sports',
       'Music', 'Film and Animation', 'Comedy', 'Nonprofits & Activism',
       'People & Blogs', 'News & Politics', 'Science & Technology',
       'Pets & Animals', 'Autos & Vehicles', 'Travel & Events', nan],
      dtype=object)

### Manually check the neirest neighbors of some selected channels

First to check how good is the embedding space, we are going to choose a channel and it's k closest channels in the embedding space. By looking at these channels in the YouTube website, we have a sense of how good/bad is the embedding.

In [7]:
# Looking at the neirest neigbors of `Shaaanxo` being a girl fashion channel
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCMpOz2KEfkSdd5JeIJh_fxw'], dict_ind_channel, k= 20, embedding_type = 'word2vecf')
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

channel     category_cc   join_date              name_cc  \
0   UCMpOz2KEfkSdd5JeIJh_fxw   Howto & Style  2009-01-18             Shaaanxo   
1   UCXbQzhqSvgVZTUyi1T4AU3w   Howto & Style  2011-08-31        Lauren Curtis   
2   UCPRlGA2w7C_DVw-1ynolJYw  People & Blogs  2011-07-19       Shaaanxo Vlogs   
3   UCLFW3EKD2My9swWH4eTLaYw   Howto & Style  2012-03-06        Chloe Morello   
4   UC21yq4sq8uxTcfgIxxyE9VQ   Howto & Style  2011-06-27          Carli Bybel   
5   UCz0Qnv6KczUe3NH1wnpmqhA   Howto & Style  2006-11-27     Nicole Guerriero   
6   UCE9_EMdxq6C8giPVpAWkJFQ   Entertainment  2011-08-10  BRITTNEYLEESAUNDERS   
7   UC0qI3HpiBua75glb4RV5mWA   Howto & Style  2013-02-21    Samantha Ravndahl   
8   UCG9nVSLp4nQlW79sY5ihRrg   Howto & Style  2013-08-17         Desi Perkins   
9   UC6jgzx2g3nlbaYkd8EMweKA   Howto & Style  2010-09-07          Jaclyn Hill   
10  UCPG6A5tNaPfv2SRNW2beq5Q   Entertainment  2011-11-13       Shani Grimmond   
11  UC85jN0RMP-uJTIzhAcMFitA  People & Blogs  2010-10-06        Karissa Pukas   
12  UC9TreTE-iXwfwQl72DzDurA   Howto & Style  2009-01-04       Kandee Johnson   
13  UCJRFJl5UDs4Citzs-uRcrxQ   Howto & Style  2011-01-12    Danielle Mansutti   
14  UCC0EqtXQ9at6ON_-ZYJaImA   Howto & Style  2014-01-26                 Katy   
15  UC3JxYPRHnB5TvaCFuKfcdJQ   Howto & Style  2012-03-02             Chrisspy   
16  UCDHQbU57NZilrhbuZNbQcRA   Howto & Style  2013-02-27        PatrickStarrr   
17  UCE1BqH-72EBs-siTbDAjOGQ   Howto & Style  2008-04-20        Marlena Stell   
18  UCEZtkE45zjun9zROKtcqOcQ   Howto & Style  2012-02-11        Amanda Ensing   
19  UCeGe5mF6zkOA17X6X1VIjkA   Howto & Style  2013-05-14           Amy Macedo   

    subscribers_cc  videos_cc  subscriber_rank_sb  weights  
0          3215340       1422              3566.0   2.2780  
1          3530000        379              3067.0   2.2335  
2           555112        742             29412.0   3.0150  
3          2700000        518              4490.0   2.3680  
4          6190000        546              1234.0   2.0870  
5          2870000        664              4160.0   2.3400  
6          1100000        584             15533.0   2.8215  
7           967843        322             17375.0   2.8485  
8          3290000        359              3358.0   2.2880  
9          5890000        344              1348.0   2.0870  
10         1590000        423              9487.0   2.5975  
11          737127        503             24009.0   2.9405  
12         3900000        630              2276.0   2.1660  
13         1610000        412              9194.0   2.5580  
14         2000000        407              6997.0   2.5275  
15         1660000        258              8889.0   2.5650  
16         4430000        358              2108.0   2.1275  
17         1520000        754              9894.0   2.6430  
18         1440000        488             10711.0   2.5920  
19          995000        696             16577.0   2.8185

In [8]:
# Looking at the neirest neigbors of `Alex Costa` being a men fashion channel
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCZyCposXwcyopaACep44maQ'], dict_ind_channel, k= 20, embedding_type = 'word2vecf')
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

channel           category_cc   join_date  \
0   UCZyCposXwcyopaACep44maQ         Howto & Style  2011-09-29   
1   UCbq8_4_mFAx_rzDF5VT7MJw         Howto & Style  2013-11-11   
2   UC5fqfItld8kQufe3djD6wAw         Howto & Style  2011-10-10   
3   UCZqsC13VeSnTIvdvTK-YEvA         Howto & Style  2012-12-01   
4   UC1PkRYud11ogYDqgdqd23Zw         Entertainment  2016-05-24   
5   UCjQG1tv6johDz6qEuaHk6QQ        People & Blogs  2014-05-18   
6   UCC-ygwC3ZfSRhIcao8a2zyQ         Howto & Style  2009-01-27   
7   UCQFmSsxwJwOALJZp3GYf7Cg         Howto & Style  2014-06-05   
8   UCQPmOWNza6PMesQaWWBEhJA        People & Blogs  2014-05-05   
9   UChNN7VBxPTiNrqjUaQd9bxA         Howto & Style  2012-09-14   
10  UC0CRYvGlWGlsGxBNgvkUbAg             Education  2012-02-07   
11  UCWYzs-Mi7SM_n1EgvzIOn3w         Entertainment  2017-01-12   
12  UCRG_IQW6Yw5JmtTlKz8PBcg        People & Blogs  2013-12-31   
13  UCFZzpLbNpQPQzhDUc3jTkLw         Entertainment  2017-01-24   
14  UCxX9wt5FWQUAAz4UrysqK9A             Education  2016-02-26   
15  UCXlnUMmbfvVBmc5NqzlbFVw                Comedy  2017-08-25   
16  UCIrn3pSjhCLp8xA7JXxi6Bw             Education  2013-07-19   
17  UCRAWGR5ySuIDNrotce8pI3w         Entertainment  2015-02-15   
18  UCEmOg9ooeiihAemFze-jQrg  Science & Technology  2015-01-28   
19  UC_1awbvccFZOnVRjAIkCG7Q  Science & Technology  2009-06-03   

                    name_cc  subscribers_cc  videos_cc  subscriber_rank_sb  \
0                Alex Costa         2120000        543              5370.0   
1                   BluMaan         1530000        381              9009.0   
2            Brett Maverick          636000        222             25157.0   
3       Mayank Bhattacharya          231121        510             74028.0   
4                Based Zeus         1280000        203             11725.0   
5         Christian Nielsen          346125        122             53608.0   
6   Slikhaar TV - Mens h...         2030000        603              6676.0   
7            Negeen Dargahi          166000        119             95742.0   
8           Anabolic Aliens          577000        723             29200.0   
9       Teachingmensfashion         4210000       1160              2093.0   
10  Gravity Transformati...         1800000        378              5749.0   
11            Jack ManleyTV         1340000         34             11112.0   
12            Austin Dunham          755000        426             21866.0   
13             Bobby Misner          203000         19             86615.0   
14                  CS Dojo         1100000         84             13133.0   
15               Jack Denmo          353000        146             36514.0   
16         MulliganBrothers          833000        374             19052.0   
17                   SinsTV         1210000        113             12269.0   
18              Anubhav Roy          187469        226             87321.0   
19              Sam Beckman          130395        206            121897.0   

    weights  
0    2.4170  
1    2.5520  
2    2.9560  
3    3.7605  
4    2.6665  
5    3.5505  
6    2.4925  
7    3.7010  
8    2.9985  
9    2.1225  
10   2.4570  
11   2.6360  
12   2.9365  
13   4.2785  
14   2.7490  
15   3.2370  
16   2.8750  
17   2.7000  
18   4.3725  
19   4.2880

In [9]:
# Looking at the neirest neigbors of `Magpiepony` which is a kids film
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCISF5OGuAtSLNF24TKTnXag'], dict_ind_channel, k= 20, embedding_type = 'word2vecf')
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

channel         category_cc   join_date  \
0   UCISF5OGuAtSLNF24TKTnXag  Film and Animation  2013-12-17   
1   UC94Z4HZJkhPm94YPH1GE3bw       Entertainment  2011-12-13   
2   UCHKDtSVVEc686EW7jRSCpZA       Entertainment  2007-04-01   
3   UC8Wj98MR_oUHBpTjLsE3HuA  Film and Animation  2010-10-10   
4   UCg3bKR3Z0rCmUe81X06EcaA       Entertainment  2015-11-21   
5   UCPflSzRRyzK2hhbVxNAnrhw  Film and Animation  2012-03-31   
6   UCcZjYWpM72s-1cinYfeSCag      People & Blogs  2010-08-05   
7   UCcGuE4cmWhB4XLjSKA8eRFA       Entertainment  2014-08-30   
8   UCaRdKsgbUdMIp-9FwEsaokg  Film and Animation  2014-02-25   
9   UChU79FLnClqlRSTSZe3EjNA  Film and Animation  2014-03-21   
10  UCVKVmfXSkqhfCIpLLXFF26w  Film and Animation  2013-09-05   
11  UCyNqz2EG8jzW1QV_YBw14xw       Entertainment  2013-09-16   
12  UCn5wtpa-WcFPeAXZumbZlyw  Film and Animation  2011-07-20   
13  UCVv1vQYLgJ6STMJkK2aWMyQ       Entertainment  2015-07-07   
14  UCV2ShQ8SuWlrSk8zBGPzwSg       Entertainment  2012-04-17   
15  UCD_VOth7RmckN6DbmFJa__A       Entertainment  2013-08-31   
16  UCtaL0UtIW2I3Iq6k0sV2lEw              Gaming  2014-03-18   
17  UCzNNpurJsoyTc6zGLnXJcPw  Film and Animation  2013-05-04   
18  UCMzfiTDOiuax4C7H1Uu88mw              Gaming  2012-02-17   
19  UC9qgVkKRZUXcgHdf35Z-8dw  Film and Animation  2013-06-24   

                    name_cc  subscribers_cc  videos_cc  subscriber_rank_sb  \
0                Magpiepony          736000        304             21120.0   
1                 EileMonty          504000        211             33178.0   
2   Scribbler Production...          327000       1767             54604.0   
3               Pinkie Rose          247000        167             72414.0   
4               Blank Slate          121000        408            129990.0   
5                Pinkie Pie         1229394        451             11840.0   
6         DisneyFanatic2364          205000       1026             88028.0   
7                 WatchPony          117000        341            144609.0   
8                  Ashley H           77200        500            196018.0   
9                Flutter525          858000       1297             20180.0   
10               Moliminous          228000        778             79008.0   
11          TheLostNarrator          115000        336            145843.0   
12                EDplus777          687592        214             24914.0   
13                Tridashie          252744         48             67004.0   
14                 StormXF3          346008         53             38030.0   
15         The Brony Notion          256132        158             70689.0   
16        MoliminousTheater          184000        251             95498.0   
17                 MLPcandy           54700        358            269490.0   
18             eric wartick          168734        103            102730.0   
19  My Little Pony Offic...          874016       1840             12248.0   

    weights  
0    2.9225  
1    3.2020  
2    3.5110  
3    3.8050  
4    4.0625  
5    2.6965  
6    4.3405  
7    4.0540  
8    5.2660  
9    2.8735  
10   3.6730  
11   4.1150  
12   2.9640  
13   4.0465  
14   3.2295  
15   3.7420  
16   3.7250  
17   5.7275  
18   3.9895  
19   2.6960

In [10]:
# Looking at neirest neigbors of `Beyond the trailer` a channel for analysis of films
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCAXR2zenCwvRIyQd9ydtfaA'], dict_ind_channel, k= 20, embedding_type = 'word2vecf')
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

channel         category_cc   join_date  \
0   UCAXR2zenCwvRIyQd9ydtfaA       Entertainment  2008-11-03   
1   UCQMbqH7xJu5aTAPQ9y_U7WQ       Entertainment  2009-03-05   
2   UCYyDbdaja1UDNdFSwUrYVGA       Entertainment  2006-02-18   
3   UC20DNxT_UjT49mYOIocJAww       Entertainment  2012-06-20   
4   UCCqEeDAUf4Mg0GgEN658tkA       Entertainment  2011-01-06   
5   UC7v3-2K1N84V67IF-WTRG-Q       Entertainment  2007-03-07   
6   UCtoMyXF4VFY3cB8fUcn7N4A       Entertainment  2006-09-19   
7   UCMVCs1F_XGueuaD9AfgTWmg       Entertainment  2008-10-20   
8   UCWvMmm_sSdgALpo1Ci4WvtQ       Entertainment  2007-02-04   
9   UCRX7UEyE8kp35mPrgC2sosA  Film and Animation  2011-06-27   
10  UCTAgbu2l6_rBKdbTvEodEDw       Entertainment  2007-06-17   
11  UCaA3Cnh8B_jmfTLX9GjIqEw       Entertainment  2011-03-20   
12  UCkDSAQ_5-yx5hmuvUcsJL7A       Entertainment  2013-08-16   
13  UCz97F7dMxBNOfGYu3rx8aCw       Entertainment  2006-06-02   
14  UCjmJDM5pRKbUlVIzDYYWb6g       Entertainment  2006-10-17   
15  UC7_WiJnJxpm4MRNesRttzWg       Entertainment  2012-10-30   
16  UCP1iRaFlS5EYjJBryFV9JPw       Entertainment  2015-11-25   
17  UCngMw0G2y2YFLPVbyEsfVog       Entertainment  2007-07-10   
18  UC2-BeLxzUBSs0uSrmzWhJuQ  Film and Animation  2006-07-13   
19  UCiifkYAs_bq1pt_zbNAzYGg       Entertainment  2011-08-19   

                    name_cc  subscribers_cc  videos_cc  subscriber_rank_sb  \
0        Beyond The Trailer          853000       5064             19744.0   
1   Fandom Entertainment...         1570000      10813              9597.0   
2               John Campea          186000       3961             83695.0   
3            ComicBookCast2          473000       7153             34911.0   
4           Chris Stuckmann         1580000       1400              8486.0   
5              Jeremy Jahns         1600000       1584              8351.0   
6              AMC Theatres          396000       9046             44969.0   
7   Movie Trivia Schmoed...          301000       3294             59054.0   
8           Collider Videos          583000       9074             30710.0   
9   JoBlo Movie Trailers...         2070000       5081              6408.0   
10                  Nerdist         2690000       2370              4427.0   
11         HN Entertainment          411000       6552             44650.0   
12         Mr Sunday Movies         1090000        745             14457.0   
13  Sony Pictures Entert...         3760000       3451              2363.0   
14  Warner Bros. Picture...         7660000       2397               779.0   
15         The Reel Rejects          682000       3996             25213.0   
16                   Looper         4820000       2533              1758.0   
17       Andre "Black Nerd"          704000       1263             23985.0   
18         20th Century Fox         3730000       2069              2606.0   
19                       DC         2380000       2587              5036.0   

    weights  
0    2.8635  
1    2.5995  
2    4.2960  
3    3.2230  
4    2.5600  
5    2.5415  
6    3.3400  
7    3.5435  
8    3.0665  
9    2.4595  
10   2.3700  
11   3.3705  
12   2.8405  
13   2.1790  
14   2.0870  
15   2.9710  
16   2.0870  
17   2.9420  
18   2.2040  
19   2.3910

### User jumper implementation

In [4]:
# We randomly choose 3 000 users and for each user we select a random pair of channel
# channels_tuple = [(channel_1, channel_2), (...)]
with open(os.path.join(COMMON_PATH, "channels_tuple_user_walk.pkl"),'rb') as f:
     channels_tuple = pickle.load(f)
f.close()

In [ ]:
files = [PATH]
users_walk_tab, users_walk_tab_new, ranking_position_tab = get_user_walk_and_position_ratio(files, channels_tuple)

In [16]:
print('User walk = ' + str(users_walk_tab))
print('Ranking position ratio = ' + str(ranking_position_tab))

[0.4285789192126741]